In [2]:
import sqlite3
import tkinter as tk
from tkinter import ttk

def execute_query(query_name):
    query_dict = {
        'Total Spent by Customers': """SELECT Customer.name, SUM(Order_.price) AS total_spent  
                                    FROM Customer
                                    JOIN Order_ ON Customer.customer_id = Order_.customer_id
                                    GROUP BY Customer.customer_id
                                    ORDER BY total_spent DESC;""", 
        'Newspapers Sold by Category': """SELECT Newspaper.category, SUM(Purchase.quantity) AS total_sold  
                                          FROM Newspaper
                                          JOIN Register ON Newspaper.newspaper_id = Register.newspaper_id  
                                          JOIN Purchase ON Register.purchase_id = Purchase.purchase_id  
                                          GROUP BY Newspaper.category;""",
        'Revenue by Payment Method': """SELECT Payment_Method.name, SUM(Order_.price) AS total_revenue
                                        FROM Payment_Method
                                        JOIN Order_ ON Payment_Method.payment_method_id = Order_.payment_method_id
                                        GROUP BY Payment_Method.name;""",
        'Average Price by Newspaper Category': """SELECT Newspaper.category, AVG(Newspaper.price) AS average_price
                                                  FROM Newspaper
                                                  GROUP BY Newspaper.category;""",
        'Top 3 Best Selling Newspapers': """SELECT Newspaper.name, SUM(Purchase.quantity) AS total_quantity_purchased
                                            FROM Newspaper
                                            JOIN Register ON Newspaper.newspaper_id = Register.newspaper_id
                                            JOIN Purchase ON Register.purchase_id = Purchase.purchase_id
                                            GROUP BY Newspaper.name
                                            ORDER BY total_quantity_purchased DESC
                                            LIMIT 3;""",
        'Customers without Orders': """SELECT Customer.*
                                       FROM Customer
                                       LEFT JOIN Order_ ON Customer.customer_id = Order_.customer_id
                                       WHERE Order_.order_id IS NULL;""",
        'Total Orders by Customer': """SELECT Customer.name, COUNT(Order_.order_id) AS total_orders
                                        FROM Customer
                                        LEFT JOIN Order_ ON Customer.customer_id = Order_.customer_id
                                        GROUP BY Customer.name;""",
        'Credit Card Payments by Order': """SELECT Order_.*
                                            FROM Order_
                                            JOIN Payment_Method ON Order_.payment_method_id = Payment_Method.payment_method_id
                                            WHERE Payment_Method.credit_card = TRUE;""",
        'Customers with Credit Card Payments': """SELECT DISTINCT Customer.*
                                                 FROM Customer
                                                 JOIN Order_ ON Customer.customer_id = Order_.customer_id
                                                 JOIN Payment_Method ON Order_.payment_method_id = Payment_Method.payment_method_id
                                                 WHERE Payment_Method.credit_card = TRUE;""",
        'Newspapers Purchased by Customer': """SELECT Newspaper.name, Newspaper.category, Purchase.quantity, Purchase.price
                                               FROM Newspaper
                                               JOIN Register ON Newspaper.newspaper_id = Register.newspaper_id
                                               JOIN Purchase ON Register.purchase_id = Purchase.purchase_id
                                               JOIN Order_ ON Purchase.order_id = Order_.order_id
                                               WHERE Order_.customer_id = 'customer_id_value';""",
        'Pending Orders': """SELECT * FROM Order_ WHERE pending = TRUE;""",
        'Cancelled Orders': """SELECT * FROM Order_ WHERE cancelled = TRUE;""",
        'Delivered Orders': """SELECT * FROM Order_ WHERE delivered = TRUE;""",
        'Large Purchases': """SELECT * FROM Purchase WHERE quantity >= 10;""",
        'Newspapers Purchased by Customer ID': """SELECT Newspaper.name, Newspaper.category
                                                  FROM Purchase
                                                  JOIN Register ON Purchase.purchase_id = Register.purchase_id
                                                  JOIN Newspaper ON Register.newspaper_id = Newspaper.newspaper_id
                                                  JOIN Order_ ON Purchase.order_id = Order_.order_id
                                                  JOIN Customer ON Order_.customer_id = Customer.customer_id
                                                  WHERE Customer.customer_id = 1;""",
        'Unfulfilled Orders by Customer ID': """SELECT Purchase.purchase_id, Purchase.quantity, Purchase.price, Order_.order_date
                                                FROM Purchase
                                                JOIN Order_ ON Purchase.order_id = Order_.order_id
                                                JOIN Customer ON Order_.customer_id = Customer.customer_id
                                                WHERE Customer.customer_id = 1 AND Order_.delivered = FALSE;"""
    }  

    query = query_dict.get(query_name)
    if query:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute(query)

        result_window = tk.Toplevel(root)
        result_window.title(f"{query_name} Results")

        header_frame = tk.Frame(result_window)
        header_frame.pack()

        for idx, column in enumerate(cursor.description):
            column_label = tk.Label(header_frame, text=column[0], relief=tk.RIDGE, padx=5, pady=5)
            column_label.grid(row=0, column=idx)

        data_frame = tk.Frame(result_window)
        data_frame.pack()

        for row_idx, row in enumerate(cursor.fetchall()):
            for col_idx, value in enumerate(row):
                value_label = tk.Label(data_frame, text=value, relief=tk.RIDGE, padx=5, pady=5)
                value_label.grid(row=row_idx, column=col_idx)

def display_query_results():
    query_name = query_selector.get()
    if query_name:
        execute_query(query_name)

def display_selected_table():
    table_name = table_selector.get()
    if table_name:
        cursor = conn.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        result_window = tk.Toplevel(root)
        result_window.title(f"{table_name} Table")

        header_frame = tk.Frame(result_window)
        header_frame.pack()

        for idx, column in enumerate(cursor.description):
            column_label = tk.Label(header_frame, text=column[0], relief=tk.RIDGE, padx=5, pady=5)
            column_label.grid(row=0, column=idx)

        data_frame = tk.Frame(result_window)
        data_frame.pack()

        for row_idx, row in enumerate(rows):
            for col_idx, value in enumerate(row):
                value_label = tk.Label(data_frame, text=value, relief=tk.RIDGE, padx=5, pady=5)
                value_label.grid(row=row_idx, column=col_idx)

# Initialize the GUI
conn = sqlite3.connect("database.db")
root = tk.Tk()
root.title("Database Viewer")

main_frame = tk.Frame(root)
main_frame.pack(padx=10, pady=10)
table_names = ['Customer', 'Newspaper', 'Payment_Method', 'Cart', 'Order_', 'Purchase', 'Register']

# Add table selection combobox and display button
table_selector = tk.StringVar()
table_combobox = ttk.Combobox(main_frame, textvariable=table_selector, values=table_names)
table_combobox.pack(padx=5, pady=5)

display_table_button = tk.Button(main_frame, text="Display Table", command=display_selected_table)
display_table_button.pack(padx=5, pady=5)

# Add query selection combobox and display button
query_names = [
    'Total Spent by Customers',
    'Newspapers Sold by Category',
    'Revenue by Payment Method',
    'Average Price by Newspaper Category',
    'Top 3 Best Selling Newspapers',
    'Customers without Orders',
    'Total Orders by Customer',
    'Credit Card Payments by Order',
    'Customers with Credit Card Payments',
    'Newspapers Purchased by Customer',
    'Pending Orders',
    'Cancelled Orders',
    'Delivered Orders',
    'Large Purchases',
    'Newspapers Purchased by Customer ID',
    'Unfulfilled Orders by Customer ID'
] 
query_selector = tk.StringVar()
query_combobox = ttk.Combobox(main_frame, textvariable=query_selector, values=query_names)
query_combobox.pack(padx=5, pady=5)

display_query_button = tk.Button(main_frame, text="Display Query Results", command=display_query_results)
display_query_button.pack(padx=5, pady=5)


# SQL code
sql_code = """
-- -- Create the Customer table
CREATE TABLE Customer (
  customer_id INTEGER PRIMARY KEY,
  phone_number TEXT NOT NULL,
  name TEXT NOT NULL,
  email TEXT,
  address TEXT
);

-- Create the Newspaper table
CREATE TABLE Newspaper (
  newspaper_id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  category TEXT NOT NULL,
  date DATE NOT NULL,
  description TEXT,
  price REAL NOT NULL
);

-- Create the Payment_Method table
CREATE TABLE Payment_Method (
  payment_method_id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  cash BOOLEAN NOT NULL,
  credit_card BOOLEAN NOT NULL
);

-- Create the Cart table
CREATE TABLE Cart (
  cart_id INTEGER PRIMARY KEY,
  customer_id INTEGER NOT NULL,
  product_name TEXT NOT NULL,
  FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
);

-- Create the Order_ table
CREATE TABLE Order_ (
  order_id INTEGER PRIMARY KEY,
  customer_id INTEGER NOT NULL,
  payment_method_id INTEGER NOT NULL,
  order_date DATE NOT NULL,
  price REAL NOT NULL,
  pending BOOLEAN NOT NULL,
  cancelled BOOLEAN NOT NULL,
  delivered BOOLEAN NOT NULL,
  FOREIGN KEY (customer_id) REFERENCES Customer (customer_id),
  FOREIGN KEY (payment_method_id) REFERENCES Payment_Method (payment_method_id)
);

-- Create the Purchase table
CREATE TABLE Purchase (
  purchase_id INTEGER PRIMARY KEY,
  quantity INTEGER NOT NULL,
  price REAL NOT NULL,
  order_id INTEGER NOT NULL,
  cart_id INTEGER NOT NULL,
  FOREIGN KEY (order_id) REFERENCES Order_ (order_id),
  FOREIGN KEY (cart_id) REFERENCES Cart (cart_id)
);

-- Create the Register table
CREATE TABLE Register (
  purchase_id INTEGER NOT NULL,
  newspaper_id INTEGER NOT NULL,
  PRIMARY KEY (purchase_id, newspaper_id),
  FOREIGN KEY (purchase_id) REFERENCES Purchase (purchase_id),
  FOREIGN KEY (newspaper_id) REFERENCES Newspaper (newspaper_id)
);

-- Insert data into the Customer table
INSERT INTO Customer (customer_id, phone_number, name, email, address)
VALUES (1, '555-1234', 'Amr', 'amr@email.com', '123 Main St'),
       (2, '555-5678', 'Jane', NULL, '456 Oak St'),
       (3, '555-9012', 'Johnson', 'bjohnson@email.com', '789 Elm St'),
       (4, '555-3456', 'Mary', NULL, '101 Maple Ave'),
       (5, '555-7890', 'Tom', 'twilson@email.com', '234 Pine St'),
       (6, '555-2345', 'Brown', NULL, '567 Cedar St');

-- Insert data into the Newspaper table
INSERT INTO Newspaper (newspaper_id, name, category, date, description, price)
VALUES (1, 'New York Times', 'National', '2023-05-06', 'Daily newspaper', 2.50),
       (2, 'Chicago Tribune', 'National', '2023-05-06', 'Daily newspaper', 2.00),
       (3, 'Los Angeles Times', 'National', '2023-05-06', 'Daily newspaper', 2.25),
       (4, 'Washington Post', 'National', '2023-05-06', 'Daily newspaper', 2.50),
       (5, 'Wall Street Journal', 'Business', '2023-05-06', 'Daily newspaper', 3.00),
       (6, 'USA Today', 'National', '2023-05-06', 'Daily newspaper', 2.50);

-- Insert data into the Payment_Method table
INSERT INTO Payment_Method (payment_method_id, name, cash, credit_card)
VALUES (1, 'Cash', TRUE, FALSE),
       (2, 'Credit Card', FALSE, TRUE),
       (3, 'Mobile Payment', FALSE, TRUE);

-- Insert data into the Cart table
INSERT INTO Cart (cart_id, customer_id, product_name)
VALUES (1, 1, 'New York Times'),
       (2, 1, 'Chicago Tribune'),
       (3, 3, 'Los Angeles Times'),
       (4, 4, 'Washington Post'),
       (5, 5, 'Wall Street Journal'),
       (6, 6, 'USA Today');

-- Insert data into the Order_ table
INSERT INTO Order_ (order_id, customer_id, payment_method_id, order_date, price, pending, cancelled, delivered)
VALUES (1, 1, 2, '2023-05-06', 6.50, FALSE, FALSE, FALSE),
       (2, 3, 1, '2023-05-06', 11.75, TRUE, FALSE, FALSE),
       (3, 5, 3, '2023-05-06', 13.50, TRUE, TRUE, FALSE);

-- Insert data into the Purchase table
INSERT INTO Purchase (purchase_id, quantity, price, order_id, cart_id)
VALUES (1, 1, 2.50, 1, 1),
       (2, 2, 4.00, 1, 2),
       (3, 3, 6.75, 2, 3),
       (4, 1, 2.50, 2, 4),
       (5, 2, 6.00, 3, 5),
       (6, 3, 7.50, 3, 6);

-- Insert data into the Register table
INSERT INTO Register (purchase_id, newspaper_id)
VALUES (1, 1),
       (1, 2),
       (2, 3),
       (2, 4),
       (3, 5),
       (3, 6);


SELECT Payment_Method.name, SUM(Order_.price) AS total_revenue
FROM Payment_Method
JOIN Order_ ON Payment_Method.payment_method_id = Order_.payment_method_id
GROUP BY Payment_Method.name;
SELECT Newspaper.category, AVG(Newspaper.price) AS average_price
FROM Newspaper
GROUP BY Newspaper.category;
SELECT Newspaper.name, SUM(Purchase.quantity) AS total_quantity_purchased
FROM Newspaper
JOIN Register ON Newspaper.newspaper_id = Register.newspaper_id
JOIN Purchase ON Register.purchase_id = Purchase.purchase_id
GROUP BY Newspaper.name
ORDER BY total_quantity_purchased DESC
LIMIT 3;
SELECT Customer.*
FROM Customer
LEFT JOIN Order_ ON Customer.customer_id = Order_.customer_id
WHERE Order_.order_id IS NULL;
SELECT Customer.name, COUNT(Order_.order_id) AS total_orders
FROM Customer
LEFT JOIN Order_ ON Customer.customer_id = Order_.customer_id
GROUP BY Customer.name;

SELECT Order_.*
FROM Order_
JOIN Payment_Method ON Order_.payment_method_id = Payment_Method.payment_method_id
WHERE Payment_Method.credit_card = TRUE;

SELECT DISTINCT Customer.*
FROM Customer
JOIN Order_ ON Customer.customer_id = Order_.customer_id
JOIN Payment_Method ON Order_.payment_method_id = Payment_Method.payment_method_id
WHERE Payment_Method.credit_card = TRUE;
SELECT Newspaper.name, Newspaper.category, Purchase.quantity, Purchase.price
FROM Newspaper
JOIN Register ON Newspaper.newspaper_id = Register.newspaper_id
JOIN Purchase ON Register.purchase_id = Purchase.purchase_id
JOIN Order_ ON Purchase.order_id = Order_.order_id
WHERE Order_.customer_id = 'customer_id_value';

SELECT * FROM Order_ WHERE pending = TRUE;
SELECT * FROM Order_ WHERE cancelled = TRUE;
SELECT * FROM Order_ WHERE delivered = TRUE;
SELECT * FROM Purchase WHERE quantity >= 10;
SELECT Newspaper.name, Newspaper.category
FROM Purchase
JOIN Register ON Purchase.purchase_id = Register.purchase_id
JOIN Newspaper ON Register.newspaper_id = Newspaper.newspaper_id
JOIN Order_ ON Purchase.order_id = Order_.order_id
JOIN Customer ON Order_.customer_id = Customer.customer_id
WHERE Customer.customer_id = 1;
SELECT Purchase.purchase_id, Purchase.quantity, Purchase.price, Order_.order_date
FROM Purchase
JOIN Order_ ON Purchase.order_id = Order_.order_id
JOIN Customer ON Order_.customer_id = Customer.customer_id
WHERE Customer.customer_id = 1 AND Order_.delivered = FALSE;
SELECT Newspaper.category, SUM(Purchase.quantity) AS total_sold
FROM Newspaper
JOIN Register ON Newspaper.newspaper_id = Register.newspaper_id
JOIN Purchase ON Register.purchase_id = Purchase.purchase_id
GROUP BY Newspaper.category;
SELECT Customer.name, SUM(Order_.price) AS total_spent
FROM Customer
JOIN Order_ ON Customer.customer_id = Order_.customer_id
GROUP BY Customer.customer_id
ORDER BY total_spent DESC

"""

conn = sqlite3.connect(':memory:')
execute_sql_commands(conn, sql_code)

root.mainloop()

NameError: name 'execute_sql_commands' is not defined